In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task
from labels_to_ids import labels_to_ids_tam
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0 )
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average= 'macro', zero_division=0)
        tr_f1score += tmp_tr_f1score
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro', zero_division=0)
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../datasets/task_a/transliterated/'

    train_data = read_task(dataset_location , split = 'tam_train_trans')
    dev_data = read_task(dataset_location , split = 'tam_dev_trans')
    #test_data = read_task(dataset_location , split = 'dev')#load test set
    labels_to_ids = labels_to_ids_tam
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['xlm-roberta-base']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + 'Tamil' + 'transliterated' + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

...

FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 1.0008831024169922
Validation loss per 100 evaluation steps: 1.0614520090051216
DEV ACC: 0.6459173387096774
DEV Precision: 0.49692030483481053
DEV Recall: 0.47177591249651196
DEV F1Score: 0.454686531712774
BEST ACCURACY -->  DEV: 0.64592
BEST PRECISION -->  DEV: 0.51557
BEST RECALL -->  DEV: 0.49712
BEST F1SCORE -->  DEV: 0.46764
TIME PER EPOCH: 26.82423800230026

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

Training epoch: 1
FINSIHED BATCH: 19 of 1115
FINSIHED BATCH: 39 of 1115
FINSIHED BATCH: 59 of 1115
FINSIHED BATCH: 79 of 1115
FINSIHED BATCH: 99 of 1115
FINSIHED BATCH: 119 of 1115
FINSIHED BATCH: 139 of 1115
FINSIHED BATCH: 159 of 1115
FINSIHED BATCH: 179 of 1115
FINSIHED BATCH: 199 of 1115
FINSIHED BATCH: 219 of 1115
FINSIHED BATCH: 239 of 1115
FINSIHED BATCH: 259 of 1115
FINSIHED BATCH: 279 of 1115
FINSIHED BATCH: 299 of 1115
FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHE

FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019

FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 0.8347823023796082
Validation loss per 100 evaluation steps: 1.1108737044995374
DEV ACC: 0.6255428039702233
DEV Precision: 0.4741130562572705
DEV Recall: 0.45193776197603985
DEV F1Score: 0.43803472605830834
BEST ACCURACY -->  DEV: 0.64646
BEST PRECISION

In [6]:
print(best_dev_acc)

0.6464601426799007


In [7]:
print(best_epoch)

9


In [8]:
print(best_dev_precision)

0.5082804810226458


In [9]:
print(best_dev_recall)

0.4905071215505329


In [10]:
print(best_dev_f1score)

0.4610408149366509


In [11]:
print(overall_list_dev_acc)

[[0.6020471464019852, 0.6189322270471465, 0.6197076612903226, 0.6445797146401985, 0.6477202233250621, 0.6345572270471465, 0.641245347394541, 0.6240500930521091, 0.61263182382134, 0.6252907878411911], [0.6115849875930521, 0.6235266749379653, 0.6378528225806451, 0.6467897022332506, 0.6380660669975186, 0.6380854528535981, 0.619009770471464, 0.6410321029776676, 0.6172456575682381, 0.6278109491315137], [0.612437965260546, 0.6208901985111662, 0.6353908188585607, 0.6478559243176178, 0.6365151985111662, 0.6254264888337469, 0.640392369727047, 0.634247053349876, 0.6130776985111662, 0.6193199441687345], [0.6090454404466501, 0.6212779156327543, 0.6231583436724566, 0.641865694789082, 0.6438236662531017, 0.6297107630272952, 0.6458397952853597, 0.614124534739454, 0.6459173387096774, 0.6282955955334987], [0.6049550248138958, 0.6285088399503722, 0.6164120657568238, 0.6368253722084368, 0.6464601426799007, 0.633626705955335, 0.6327931141439207, 0.6361856389578163, 0.6255428039702233, 0.6124573511166252]]

In [12]:
print(overall_list_dev_precision)

[[0.32811382122435784, 0.47474938553785395, 0.45753535593088235, 0.5025597195387721, 0.5021558701444824, 0.49028785865480173, 0.5138285630794761, 0.4605328773541247, 0.47330350269862054, 0.4856695186861643], [0.4790127354141483, 0.476006882570119, 0.473671927237915, 0.4982195030197505, 0.45551855263329716, 0.4915561277435499, 0.47326458782127223, 0.498008329221576, 0.478211135425112, 0.5028067320149417], [0.4528642557664946, 0.4477187630757839, 0.49007284481406105, 0.4921765087792947, 0.4923529666401086, 0.47722255911091976, 0.5010848128554916, 0.48841815148968976, 0.46723287007435843, 0.4908428243452909], [0.36317263594108773, 0.46382027349790633, 0.47109445667842587, 0.4934303039626661, 0.48462751774899926, 0.4967065057572902, 0.51556560234821, 0.4865394864864968, 0.49692030483481053, 0.4830850381496476], [0.4292085767113139, 0.46154020202965723, 0.4751599785766578, 0.49150698276491794, 0.5082804810226458, 0.4881607696933797, 0.4835660661899265, 0.48788275529310215, 0.474113056257270

In [13]:
print(overall_list_dev_recall)

[[0.31245709492738416, 0.4664311162689448, 0.46418312125059324, 0.45065280683233633, 0.4444667278740046, 0.477969183537159, 0.45618810889121053, 0.46633481995435216, 0.47601328840336454, 0.48987535317583847], [0.4371565731578716, 0.44477614591356646, 0.424934423376083, 0.45594595561732576, 0.4258138764646487, 0.4830529451408618, 0.4889782669789056, 0.4786540152406925, 0.49493054843289197, 0.48202021471285217], [0.4164760140267736, 0.4175922108077983, 0.4593452235452378, 0.43926983018384974, 0.47044589452267216, 0.499355788781279, 0.46128870052378057, 0.47228146542591914, 0.4743746538365415, 0.4766167399230756], [0.3370850803338947, 0.4357773582307966, 0.4496335853898156, 0.4558720024130812, 0.41908474093152753, 0.47245414086646637, 0.45900766825419387, 0.4971204848935131, 0.47177591249651196, 0.44641337534185815], [0.41339298145602044, 0.41263183620785593, 0.4731809245645378, 0.44371660247412126, 0.43672966922597845, 0.476684264929906, 0.48531483511118756, 0.4725179237177281, 0.4519377

In [14]:
print(overall_list_dev_f1score)

[[0.27930099712446327, 0.44012944408140137, 0.434971125840416, 0.4449970242757829, 0.4401118025502861, 0.45834874016350097, 0.45436498874272685, 0.43812777657311663, 0.4491071526421451, 0.46377191572696774], [0.4263784391109538, 0.43364146203882314, 0.4142962475335055, 0.4431426315360798, 0.4134024844897988, 0.457213484189592, 0.4564626409109078, 0.4621834202119322, 0.46066702569648377, 0.4657449885451485], [0.40369935975859256, 0.397350625139778, 0.4468242871775994, 0.4343212999300945, 0.45467012758678726, 0.4608770677197996, 0.4491832974325184, 0.4541175576162619, 0.441418706598005, 0.4548091997600329], [0.309430311157726, 0.4195983992123217, 0.43230906431001415, 0.4436310982230985, 0.41244622002346637, 0.4594782577344092, 0.4554314296043864, 0.4676430052486152, 0.454686531712774, 0.4386316954728498], [0.3930645049547903, 0.40628854774540585, 0.45188062400866863, 0.43847573972476633, 0.43495139625772766, 0.4537535930163508, 0.45994085051548134, 0.454062758097028, 0.43803472605830834,

In [15]:
#The best model is 0